In [ ]:
import os
import pandas as pd
import re

In [ ]:
target = "RawNepQuake15"
file_list = [os.path.join(target,x) for x in os.listdir(target)]
file_list.sort()
file_list

In [ ]:
#### CLEANING ##### remove @usernames and external-urls
def cleanText(pdf):
    pdf['Text'] = pdf.apply(lambda x: re.sub('@\w+','',x['Text']).strip(),axis=1) # remove @usernames
    pdf['Text'] = pdf.apply(lambda x: re.sub('#\w+','',x['Text']).strip(),axis=1) # remove hash-tag
    pdf['Text'] = pdf.apply(lambda x: re.sub('\w+:\/\/[\w.\/]+',' ',x['Text']).strip(),axis=1)  # remove external urls
    pdf['Text'] = pdf.apply(lambda x: re.sub('[a-zA-Z]+','',x['Text']).strip(),axis=1)  # remove english alphabets
    # pdf['Text'] = pdf.apply(lambda x: re.sub('\s+',' ',x['Text']).strip(),axis=1) # remove new line feeds and extra spaces
    return pdf


In [ ]:
agg = pd.DataFrame()

In [ ]:
for file in file_list:
    print(file)
    pdf = pd.read_csv(file)
    print('before',pdf.shape)
    pdf = cleanText(pdf)
    
    pdf.index = range(pdf.shape[0])
    
    ### Checking for shortest string in the dataframe
    lengths = pdf.apply(lambda x: len(x['Text'].split(" ")),axis=1)
    print("Min",min(lengths),"Max", max(lengths),"Avg",sum(lengths)/lengths.shape[0])
    
    ##### CLEANING ##### remove short sentences ----
    indexes = pdf.apply(lambda x: len(x['Text'].split(" "))>=5,axis=1)
    pdf = pdf[indexes]
    
    ### Drop Duplicates
    pdf = pdf.drop_duplicates()
    print("after",pdf.shape)
    
    if not agg.empty:
        agg = pd.concat([agg,pdf])
    else:
        agg = pdf
    
agg = agg.drop_duplicates()

In [ ]:
agg['Date'].value_counts(), agg.shape

In [60]:
agg.to_csv(target+"/aggregated.csv",index=False)